# **Generate Korean DPR Dataset (KorQaud v1)**

- KorQuad v1 데이터 셋을 이용해 DPR 모델을 학습하기 위한 데이터 셋을 구축하는 코드입니다.
- 단답형 정답이 없는 데이터 셋은 **'generate_dataset_alpaca.ipynb'** 파일을 참고해주세요. 
- Chunking은 nltk 라이브러리의 sent_tokenize 를 이용해 문장 단위로 이루어집니다.  
- 이 코드로 구성한 최종 데이터 셋은 다음과 같은 조건을 만족해야 합니다. (코드를 따라가면 자동으로 충족됩니다.)
  - 모든 데이터 셋은 정해진 템플릿의 구성을 따라야 합니다.
  - 각 텍스트(text)는 고유한 인덱스 가져야 합니다.
  - Hard Negative 샘플을 이용할 경우 그 수는 텍스트의 수와 일치해야 합니다.

In [1]:
import json
import pandas as pd
from tqdm import tqdm
from nltk import sent_tokenize
from rank_bm25 import BM25Okapi
from datasets import load_dataset
from transformers import AutoTokenizer

### 1. Load Dataset

- KorQuad v1 데이터 셋을 로드합니다.
- title, context, question, answers 등의 정보를 확인합니다.
- 다른 데이터 셋을 이용할 경우 형식을 맞추거나 코드를 수정해서 사용하시면 됩니다.

In [2]:
dataset = load_dataset('KorQuAD/squad_kor_v1', trust_remote_code=True)

In [3]:
dataset['train'][1]

{'id': '6566495-0-1',
 'title': '파우스트_서곡',
 'context': '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.',
 'question': '바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가?',
 'answers': {'text': ['1악장'], 'answer_start': [421]}}

### 2. Chunk Contexts

- nltk의 sent_tokenize를 이용해 context를 chunk 단위로 분할합니다.
- **chunk_context** 함수에서 num_sents는 한 chunk 당 문장의 수이고, overlaps는 연속된 chunk 간 겹치는 문장의 수입니다.
- chunking 을 원하지 않을 경우 100 처럼 아주 큰 수를 입력하면 됩니다.
- 두 변수를 원하는 값으로 설정한 후 chunk가 잘 만들어지는지 확인합니다. 

In [4]:
def chunk_context(context, num_sents, overlaps):
    chunks = []
    start, end = 0, num_sents
    
    total_sents = sent_tokenize(context)

    while start < len(total_sents):
        chunk = total_sents[start:end]
        chunks.append(' '.join(chunk))
        
        start += (num_sents - overlaps)
        end = start + num_sents

    return chunks

In [5]:
sample_ctx = dataset['train'][0]['context']

chunks = chunk_context(sample_ctx, 3, 1)
chunks

['1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다.',
 '또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다.',
 '그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다.',
 '결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.']

### 3. Fill Templates

- 아래는 DPR 모델 학습을 위한 템플릿입니다.
- TEMPATE_ALL 의 positive 키의 리스트에 TEMPLATE_TEXT가 요소로 들어가는 구성입니다.
- __fill_templates__ 함수는 여러 개의 chunk 중 단답형 정답을 포함하는 chunk 만 템플릿 구성에 사용합니다.
- 앞서 구성한 chunks와 아래의 title, question, answer 를 이용해 템플릿을 잘 구성하는지 확인합니다.
- 사용하는 데이터 셋에 **제목(title)이 없으면 그대로 "" 로 두면 됩니다**. None 값이 잘못 들어갈 경우 학습 과정에서 오류가 발생할 수 있습니다.  
- 완성된 템플릿의 'answers(예: \[교향곡\])' 는 템플릿 구성 과정에서만 필요한 것으로 이후 없어도 무관합니다.

In [6]:
TEMPLATE_ALL = {
    "question": "",
	"answers": [],
	"positive": [],
  }

TEMPLATE_TEXT = {
    "title": "",
    "text": "",
}

In [7]:
title = dataset['train'][0]['title']
title

'파우스트_서곡'

In [8]:
question = dataset['train'][0]['question']
question

'바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?'

In [9]:
answer = dataset['train'][0]['answers']['text'][0]
answer

'교향곡'

In [10]:
def fill_templates(chunks, title, question, answer):
    positives = []
    for chunk in chunks:
        template_instance1 = TEMPLATE_TEXT.copy()
        template_instance1['title'] = title
        template_instance1['text'] = chunk
    
        if answer in chunk:
            positives.append(template_instance1.copy())

    template_instance2 = TEMPLATE_ALL.copy()
    template_instance2['question'] = question
    template_instance2['answers'] = [answer]
    template_instance2['positive'] = positives
    
    return template_instance2

In [11]:
fill_templates(chunks, title, question, answer)

{'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?',
 'answers': ['교향곡'],
 'positive': [{'title': '파우스트_서곡',
   'text': '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다.'},
  {'title': '파우스트_서곡',
   'text': '또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다.'},
  {'title': '파우스트_서곡',
   'text': '그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다.'}]}

### 4. Construct DPR Dataset

- 이상의 **chunk_context** 함수와 **fill_templates** 함수를 종합해 전체 데이터 셋에 적용합니다.
- 이를 수행하는 함수가 **construct_dpr_dataset** 입니다.
- 아래처럼 원하는 num_sents와 overlaps 값을 설정해 전체 데이터 셋에 함수를 적용합니다.

In [12]:
TEMPLATE_ALL = {
    "question": "",
	"answers": [],
	"positive": [],
  }

TEMPLATE_TEXT = {
    "title": "",
    "text": "",
}

In [13]:
def construct_dpr_dataset(dataset, num_sents, overlaps):
    dpr_dataset = []
    for idx in tqdm(range(len(dataset))):
        sample = dataset[idx]
        
        title = sample['title']
        question = sample['question']
        answer = sample['answers']['text'][0]
        chunks = chunk_context(sample['context'], num_sents, overlaps)
        
        output = fill_templates(chunks, title, question, answer)
        dpr_dataset.append(output)
    
    return dpr_dataset

In [14]:
dpr_dataset = construct_dpr_dataset(dataset['train'], 3, 1)

100%|███████████████████████████████████████████████████████████████████████████| 60407/60407 [00:13<00:00, 4565.82it/s]


In [21]:
dpr_dataset[0]

{'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?',
 'answers': ['교향곡'],
 'positive': [{'title': '파우스트_서곡',
   'text': '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다.',
   'idx': 0},
  {'title': '파우스트_서곡',
   'text': '또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다.',
   'idx': 1},
  {'title': '파우스트_서곡',
   'text': '그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다.',
   'idx': 2}],
 'an

- 코드 실행 결과, 위와 같은 완성된 템플릿이 담긴 리스트가 반환됩니다.
- **만일 여러 데이터 셋을 종합해서 사용할 경우, 이 단계에서 각 데이터 셋의 리스트를 하나로 통합해주면 됩니다.**
- 다음 단계에서 각 텍스트마다 고유한 인덱스를 부여하게 되므로 반드시 이 단계에서 통합해주어야 합니다. 

### 5. Set Index

- 데이터 셋의 각 텍스트에 고유한 인덱스를 부여합니다.
- 데이터 셋에 동일한 텍스트가 여러 개 있을 경우 모두 동일한 인덱스를 부여받게 됩니다.
- **text_2_index와 text_2_title** 은 **5.Add Hard Negative** 에서 이용합니다.

In [16]:
def set_index_to_text(dataset):
    text_2_index = {}
    text_2_title = {}
    current_idx = 0  # Initialize the starting index

    for sample in dataset:
        pos_text = [pos['text'] for pos in sample['positive']]
        pos_title = [pos['title'] for pos in sample['positive']]
    
        all_idx = []
        for text, title in zip(pos_text, pos_title):
            if text not in text_2_index:
                text_2_index[text] = current_idx 
                text_2_title[text] = title  # Map the text to its title
                current_idx += 1

        for pos in sample['positive']:
            pos['idx'] = text_2_index[pos['text']]
            all_idx.append(text_2_index[pos['text']])
        
        sample['answer_idx'] = all_idx

    return dataset, text_2_index, text_2_title

In [17]:
dpr_dataset, text_2_index, text_2_title = set_index_to_text(dpr_dataset)

In [18]:
dpr_dataset[0]

{'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?',
 'answers': ['교향곡'],
 'positive': [{'title': '파우스트_서곡',
   'text': '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다.',
   'idx': 0},
  {'title': '파우스트_서곡',
   'text': '또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다.',
   'idx': 1},
  {'title': '파우스트_서곡',
   'text': '그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다.',
   'idx': 2}],
 'an

- 이상으로 DPR 모델을 학습할 수 있는 데이터 셋이 완성되었습니다.
- Hard Negative 샘플을 추가해 모델의 성능을 향상시키고 싶으면 **5.Add Hard Netgative** 로 가면됩니다.
- Hard Negative 샘플 없이 데이터를 저장하고 싶으면 **6.Save to Json** 으로 가면됩니다.
- **Validation Set의 경우 Hard Negative 샘플 없이 바로 저장하면 됩니다.**

### 5. Add Hard Negative (Optional)

- **get_hard_negative** 함수를 이용해 **'정답이 아니지만 정답처럼 보이는 텍스트(Hard Negative)'** 를 찾습니다. 
- BM25 점수가 가장 높은 **n** 개를 추출한 후 그 중에서 **가장 높은 점수**의 텍스트가 차례로 Hard Negative 샘플로 선택됩니다. 
- KorQuad의 경우 여러 질문이 동일한 텍스트에 연결된 경우가 많은데, False Negative를 방지하기 위해 동일한 텍스트는 모두 필터링합니다.
- 필터링의 결과로 n 개의 텍스트에서 Hard Negative 샘플을 정해진 수만큼 찾지 못한 경우 **기존의 것을 복제**해 사용합니다.
- 따라서 넉넉한 n 값을 설정하는 것이 좋습니다. 다만, n 값이 커질수록 데이터 셋 구축 시간이 오래걸립니다.
- 함수 실행 결과, **positive의 텍스트 개수와 hard_neg의 텍스트 개수는 일치해야 합니다**.  

In [19]:
def get_hard_negative(dataset, text_2_index, text_2_title, n=30):
    corpus = list(text_2_index.keys())
    tokenized_corpus = [context.split(" ") for context in corpus]
    bm25 = BM25Okapi(tokenized_corpus)

    for sample in tqdm(dataset, desc="Processing samples"):
        question = sample['question']
        positive = sample['positive']

        top_n = bm25.get_top_n(question.split(" "), corpus, n=n)  # Increase n to have more candidates
        
        hard_neg = []
        positive_idx_set = set(sample['answer_idx'])
        for doc in top_n:
            if text_2_index[doc] not in positive_idx_set:
                hard_neg.append({'title': text_2_title[doc],
                                 'text': doc,
                                 'idx': text_2_index[doc]})
            
            if len(hard_neg) >= len(positive):
                break

        if len(hard_neg) == 0:
            print(f"Skipping sample due to lack of hard negatives: {question}")
            continue
        
        # If not enough hard negatives, repeat existing ones to match the number of positives
        if len(hard_neg) < len(positive):
            print(f"Warning: Not enough hard negatives for question: {question}. Duplicating existing hard negatives.")
            while len(hard_neg) < len(positive):
                hard_neg.extend(hard_neg[:len(positive) - len(hard_neg)])
        
        sample['hard_neg'] = hard_neg[:len(positive)]  # Trim to ensure exact number matches positive samples
        
    return dataset

In [20]:
dpr_dataset = get_hard_negative(dpr_dataset, text_2_index, text_2_title, n=30)

Processing samples: 100%|█████████████████████████████████████████████████████████| 60407/60407 [37:54<00:00, 26.56it/s]


In [22]:
dpr_dataset[0]

{'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?',
 'answers': ['교향곡'],
 'positive': [{'title': '파우스트_서곡',
   'text': '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다.',
   'idx': 0},
  {'title': '파우스트_서곡',
   'text': '또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다.',
   'idx': 1},
  {'title': '파우스트_서곡',
   'text': '그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다.',
   'idx': 2}],
 'an

### 6. Save to Json

- 이상의 방법으로 구축한 데이터 셋을 json 파일로 저장합니다.
- 다시 로드해봄으로써 문제 없이 로드되는지 확인합니다.

In [23]:
file_path = 'korquad_v1_train.json'

In [24]:
with open(file_path, 'w') as outfile:
    json.dump(dpr_dataset, outfile, indent=4)

In [25]:
with open(file_path) as infile:
    dataset = json.load(infile)

In [26]:
dataset[0]

{'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?',
 'answers': ['교향곡'],
 'positive': [{'title': '파우스트_서곡',
   'text': '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다.',
   'idx': 0},
  {'title': '파우스트_서곡',
   'text': '또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다.',
   'idx': 1},
  {'title': '파우스트_서곡',
   'text': '그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다.',
   'idx': 2}],
 'an